# cs5044 final project experiment 2a: predicting over entire dataset

### below are our two experiments for neural networks with 5 hidden layers

In [1]:
%matplotlib inline
import math
import pandas as pd
from pandas.plotting import lag_plot
from IPython.display import display
from sklearn import preprocessing
from sklearn import linear_model, neighbors
from sklearn import tree
from sklearn import model_selection
from sklearn import metrics
from sklearn.metrics import mean_squared_error
from sklearn.neural_network import MLPRegressor


import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()

# preprocessing

### read in data

In [2]:
df_index = pd.read_csv('sp500_index.csv')
df_stocks = pd.read_csv('sp500_stocks.csv')
df_companies = pd.read_csv('sp500_companies.csv')

### convert to datetime objects

In [3]:
df_stocks['Date'] = pd.to_datetime(df_stocks['Date'], utc=True, errors='coerce')
df_stocks['Date'] = pd.DatetimeIndex(df_stocks['Date']).date

df_index['Date'] = pd.to_datetime(df_index['Date'], utc=True)
df_index['Date'] = pd.DatetimeIndex(df_index['Date']).date

### merge

In [4]:
df = df_companies.merge(df_stocks, on='Symbol', how='inner')
df = df.merge(df_index, on='Date')
df.head()

,Exchange,Symbol,Shortname,Longname,Sector,Industry,Currentprice,Marketcap,Ebitda,Revenuegrowth,...,Longbusinesssummary,Weight,Date,Adj Close,Close,High,Low,Open,Volume,S&P500
0,NMS,AAPL,Apple Inc.,Apple Inc.,Technology,Consumer Electronics,150.43,2468014718976,1.295570e+11,0.019,...,"Apple Inc. designs, manufactures, and markets ...",0.084154,2012-09-24,21.305809,24.671070,24.825714,24.392857,24.530714,639766400.0,1456.89
1,NMS,MSFT,Microsoft Corporation,Microsoft Corporation,Technology,Software—Infrastructure,237.92,1786298499072,9.798300e+10,0.124,...,"Microsoft Corporation develops, licenses, and ...",0.060909,2012-09-24,25.504580,30.780001,31.070000,30.639999,31.000000,46825900.0,1456.89
2,NYQ,BRK-B,Berkshire Hathaway Inc. New,Berkshire Hathaway Inc.,Financial Services,Insurance—Diversified,267.77,599609245696,1.177540e+11,0.096,...,"Berkshire Hathaway Inc., through its subsidiar...",0.020445,2012-09-24,88.660004,88.660004,89.669998,88.650002,88.940002,4474500.0,1456.89
3,NYQ,UNH,UnitedHealth Group Incorporated,UnitedHealth Group Incorporated,Healthcare,Healthcare Plans,513.61,483744088064,2.756900e+10,0.126,...,UnitedHealth Group Incorporated operates as a ...,0.016495,2012-09-24,48.211803,55.980000,56.639999,55.849998,56.080002,6443800.0,1456.89
4,NYQ,JNJ,Johnson & Johnson,Johnson & Johnson,Healthcare,Drug Manufacturers—General,166.72,438907076608,3.238200e+10,0.030,...,"Johnson & Johnson, together with its subsidiar...",0.014966,2012-09-24,53.124275,69.000000,69.199997,68.809998,68.879997,9082100.0,1456.89


### drop irrelevant columns

In [5]:
df.drop(labels=['Shortname', 'Longname', 'City', 'Longbusinesssummary', 'Date'], axis=1, inplace=True)

### one hot encode 

In [6]:
ohe = preprocessing.OneHotEncoder(dtype=int, sparse=False, handle_unknown="ignore")
data = ohe.fit_transform(df[['Exchange', 'Sector', 'Industry', 'State', 'Country']])
cats = pd.DataFrame(data, columns=ohe.get_feature_names())
snp_df = pd.concat([cats, df], axis=1)
snp_df.drop(columns=['Exchange', 'Sector', 'Industry', 'State', 'Country'], inplace=True)

snp_df

C:\Users\harge\anaconda3\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


,x0_BTS,x0_NGM,x0_NMS,x0_NYQ,x1_Basic Materials,x1_Communication Services,x1_Consumer Cyclical,x1_Consumer Defensive,x1_Energy,x1_Financial Services,...,Revenuegrowth,Fulltimeemployees,Weight,Adj Close,Close,High,Low,Open,Volume,S&P500
0,0,0,1,0,0,0,0,0,0,0,...,0.019,154000.0,0.084154,21.305809,24.671070,24.825714,24.392857,24.530714,639766400.0,1456.89
1,0,0,1,0,0,0,0,0,0,0,...,0.124,221000.0,0.060909,25.504580,30.780001,31.070000,30.639999,31.000000,46825900.0,1456.89
2,0,0,0,1,0,0,0,0,0,1,...,0.096,372000.0,0.020445,88.660004,88.660004,89.669998,88.650002,88.940002,4474500.0,1456.89
3,0,0,0,1,0,0,0,0,0,0,...,0.126,350000.0,0.016495,48.211803,55.980000,56.639999,55.849998,56.080002,6443800.0,1456.89
4,0,0,0,1,0,0,0,0,0,0,...,0.030,141700.0,0.014966,53.124275,69.000000,69.199997,68.809998,68.879997,9082100.0,1456.89
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1240876,0,0,1,0,0,0,0,1,0,0,...,-0.065,32000.0,0.000212,14.630000,14.630000,14.660000,14.190000,14.630000,3855500.0,3693.23
1240877,0,0,0,1,0,0,0,0,0,0,...,-0.086,130000.0,0.000205,23.850000,23.850000,25.920000,23.430000,25.600000,5275000.0,3693.23
1240878,0,0,0,1,0,0,1,0,0,0,...,167.368,34700.0,0.000193,13.550000,13.550000,14.000000,13.120000,13.960000,25406400.0,3693.23
1240879,0,0,0,1,0,0,0,0,0,0,...,0.741,22092.0,0.000176,40.779999,40.779999,41.340000,40.099998,41.150002,1672000.0,3693.23


### check % of missing values

In [8]:
pd.options.display.max_columns = None
pd.options.display.max_rows = 250
percent_missing = snp_df.isnull().sum() * 100 / len(snp_df)
missing_value_df = pd.DataFrame({'column_name': snp_df.columns,
'percent_missing': percent_missing})

missing_value_df

,column_name,percent_missing
x0_BTS,x0_BTS,0.000000
x0_NGM,x0_NGM,0.000000
x0_NMS,x0_NMS,0.000000
x0_NYQ,x0_NYQ,0.000000
x1_Basic Materials,x1_Basic Materials,0.000000
x1_Communication Services,x1_Communication Services,0.000000
x1_Consumer Cyclical,x1_Consumer Cyclical,0.000000
x1_Consumer Defensive,x1_Consumer Defensive,0.000000
x1_Energy,x1_Energy,0.000000
x1_Financial Services,x1_Financial Services,0.000000


### drop missing values

In [9]:
snp_df.dropna(inplace=True)
snp_df

,x0_BTS,x0_NGM,x0_NMS,x0_NYQ,x1_Basic Materials,x1_Communication Services,x1_Consumer Cyclical,x1_Consumer Defensive,x1_Energy,x1_Financial Services,x1_Healthcare,x1_Industrials,x1_Real Estate,x1_Technology,x1_Utilities,x2_Advertising Agencies,x2_Aerospace & Defense,x2_Agricultural Inputs,x2_Airlines,x2_Apparel Manufacturing,x2_Apparel Retail,x2_Asset Management,x2_Auto & Truck Dealerships,x2_Auto Manufacturers,x2_Auto Parts,x2_Banks—Diversified,x2_Banks—Regional,x2_Beverages—Brewers,x2_Beverages—Non-Alcoholic,x2_Beverages—Wineries & Distilleries,x2_Biotechnology,x2_Building Materials,x2_Building Products & Equipment,x2_Business Equipment & Supplies,x2_Capital Markets,x2_Chemicals,x2_Communication Equipment,x2_Computer Hardware,x2_Confectioners,x2_Conglomerates,x2_Consulting Services,x2_Consumer Electronics,x2_Copper,x2_Credit Services,x2_Diagnostics & Research,x2_Discount Stores,x2_Drug Manufacturers—General,x2_Drug Manufacturers—Specialty & Generic,x2_Electronic Components,x2_Electronic Gaming & Multimedia,x2_Engineering & Construction,x2_Entertainment,x2_Farm & Heavy Construction Machinery,x2_Farm Products,x2_Financial Data & Stock Exchanges,x2_Food Distribution,x2_Footwear & Accessories,"x2_Furnishings, Fixtures & Appliances",x2_Gold,x2_Grocery Stores,x2_Healthcare Plans,x2_Home Improvement Retail,x2_Household & Personal Products,x2_Industrial Distribution,x2_Information Technology Services,x2_Insurance Brokers,x2_Insurance—Diversified,x2_Insurance—Life,x2_Insurance—Property & Casualty,x2_Insurance—Reinsurance,x2_Insurance—Specialty,x2_Integrated Freight & Logistics,x2_Internet Content & Information,x2_Internet Retail,x2_Leisure,x2_Lodging,x2_Luxury Goods,x2_Medical Care Facilities,x2_Medical Devices,x2_Medical Distribution,x2_Medical Instruments & Supplies,x2_Oil & Gas E&P,x2_Oil & Gas Equipment & Services,x2_Oil & Gas Integrated,x2_Oil & Gas Midstream,x2_Oil & Gas Refining & Marketing,x2_Packaged Foods,x2_Packaging & Containers,x2_Personal Services,x2_Pharmaceutical Retailers,x2_REIT—Diversified,x2_REIT—Healthcare Facilities,x2_REIT—Hotel & Motel,x2_REIT—Industrial,x2_REIT—Office,x2_REIT—Residential,x2_REIT—Retail,x2_REIT—Specialty,x2_Railroads,x2_Real Estate Services,x2_Rental & Leasing Services,x2_Residential Construction,x2_Resorts & Casinos,x2_Restaurants,x2_Scientific & Technical Instruments,x2_Security & Protection Services,x2_Semiconductor Equipment & Materials,x2_Semiconductors,x2_Software—Application,x2_Software—Infrastructure,x2_Solar,x2_Specialty Business Services,x2_Specialty Chemicals,x2_Specialty Industrial Machinery,x2_Specialty Retail,x2_Staffing & Employment Services,x2_Steel,x2_Telecom Services,x2_Tobacco,x2_Tools & Accessories,x2_Travel Services,x2_Trucking,x2_Utilities—Diversified,x2_Utilities—Independent Power Producers,x2_Utilities—Regulated Electric,x2_Utilities—Regulated Gas,x2_Utilities—Regulated Water,x2_Waste Management,x3_AL,x3_AR,x3_AZ,x3_CA,x3_CO,x3_CT,x3_DC,x3_DE,x3_FL,x3_GA,x3_IA,x3_ID,x3_IL,x3_IN,x3_KY,x3_LA,x3_MA,x3_MD,x3_ME,x3_MI,x3_MN,x3_MO,x3_NC,x3_NE,x3_NJ,x3_NV,x3_NY,x3_OH,x3_OK,x3_OR,x3_PA,x3_RI,x3_TN,x3_TX,x3_UT,x3_VA,x3_WA,x3_WI,x3_nan,x4_Bermuda,x4_Ireland,x4_Israel,x4_Netherlands,x4_Switzerland,x4_United Kingdom,x4_United States,Symbol,Currentprice,Marketcap,Ebitda,Revenuegrowth,Fulltimeemployees,Weight,Adj Close,Close,High,Low,Open,Volume,S&P500
0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,AAPL,150.43,2468014718976,1.295570e+11,0.019,154000.0,0.084154,21.305809,24.671070,24.825714,24.392857,24.530714,639766400.0,1456.89
1,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,

### separate target and features

In [10]:
snp_xvalues = snp_df.drop(["Adj Close", "Close", "High", "Low", "Open", "Symbol"], axis=1)
snp_target = snp_df["Adj Close"]

# neural network

In [11]:
X = snp_xvalues.to_numpy()
y = snp_target.to_numpy()

## activation='relu'

### layers=5, size=100

In [13]:
kfold = model_selection.KFold(5, shuffle=True, random_state=2)

r2= []
RMSE = []

for train_idx, test_idx in kfold.split(snp_df[:200000]):
    X_train, X_test = X[train_idx], X[test_idx]
    y_train, y_test = y[train_idx], y[test_idx]
    
    mlp_regr = MLPRegressor(hidden_layer_sizes=(100, 100, 100, 100, 100), max_iter=2000, activation="relu", random_state=2)
    X_scaler = preprocessing.MinMaxScaler()
    
    X_train = X_scaler.fit_transform(X_train)    
    
    mlp_regr.fit(X_train, y_train)
    
    X_test = X_scaler.transform(X_test)
    y_pred = mlp_regr.predict(X_test)
    
    r2 += [metrics.r2_score(y_test, y_pred)]
    RMSE += [math.sqrt(metrics.mean_squared_error(y_test, y_pred))]

print("r2    = {:.4f} ±{:.4f}".format(np.mean(r2), np.std(r2)))
print("RMSE = {:.4f} ±{:.4f}".format(np.mean(RMSE), np.std(RMSE)))

r2    = 0.9955 ±0.0008
RMSE = 5.2639 ±0.4863


## activation='tanh'

### layers=5, size=100

In [14]:
kfold = model_selection.KFold(5, shuffle=True, random_state=2)

r2= []
RMSE = []

for train_idx, test_idx in kfold.split(snp_df[:200000]):
    X_train, X_test = X[train_idx], X[test_idx]
    y_train, y_test = y[train_idx], y[test_idx]
    
    mlp_regr = MLPRegressor(hidden_layer_sizes=(100, 100, 100, 100, 100), max_iter=2000, activation="tanh", random_state=2)
    X_scaler = preprocessing.MinMaxScaler()
    
    X_train = X_scaler.fit_transform(X_train)    
    
    mlp_regr.fit(X_train, y_train)
    
    X_test = X_scaler.transform(X_test)
    y_pred = mlp_regr.predict(X_test)
    
    r2 += [metrics.r2_score(y_test, y_pred)]
    RMSE += [math.sqrt(metrics.mean_squared_error(y_test, y_pred))]

print("r2    = {:.4f} ±{:.4f}".format(np.mean(r2), np.std(r2)))
print("RMSE = {:.4f} ±{:.4f}".format(np.mean(RMSE), np.std(RMSE)))

r2    = 0.9914 ±0.0072
RMSE = 6.8600 ±2.6870
